# Federated datasets

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/fedjax/blob/main/docs/notebooks/dataset_tutorial.ipynb)

This tutorial introduces datasets in FedJAX and how to work with them. By completing this tutorial, we'll learn about the best practices for working with datasets.

**NOTE: For datasets, we operate over NumPy arrays NOT JAX arrays.**

In [ ]:
# Uncomment these to install fedjax.
# !pip install fedjax
# !pip install --upgrade git+https://github.com/google/fedjax.git
# !pip install tensorflow_datasets

In [ ]:
import functools
import itertools
import fedjax
import numpy as np

## What are datasets in federated learning?

In the context of federated learning (FL), data is decentralized across clients, with each client having their own local set of examples. In light of this, we refer to two levels of organization for datasets:

- Federated dataset: A collection of clients, each with their own local datasets and metadata
- Client dataset: The set of local examples for a particular client

We can think of federated data as a mapping from client ids to client datasets and client datasets as a list of examples.

```
federated_data = {
  'client0': ['a', 'b', 'c'],
  'client1': ['d', 'e'],
}
```

### Federated datasets

`fedjax.FederatedData` is the interface for accessing all federated datasets.

FedJAX comes packaged with multiple federated datasets, and we will look at the Shakespeare dataset as an example. The Shakespeare dataset is created from [The Complete Works of Shakespeare](https://www.gutenberg.org/files/100/100-0.txt), by treating each character in the play as a "client", and their dialogue lines as the examples.

FedJAX organizes federated datasets as Python modules. `load_data()` from a A dataset module loads all predefined splits together. In the case of the Shakespeare dataset, `load_data()` returns two splits: train and test.

In [ ]:
# We cap max sentence length to 8.
train_fd, test_fd = fedjax.datasets.shakespeare.load_data(sequence_length=8)

`fedjax.FederatedData` provides methods for accessing metadata about the federated dataset, like the total number of clients, client ids, and number of examples for each client.

As seen in the output, there are 715 total clients in the Shakespeare dataset.
Each client has a unique client ID that can be used to query metadata about that client such as the number of examples that client has.

In [ ]:
print('num_clients =', train_fd.num_clients())

# train_fd.client_ids() is a generator of client ids.
# itertools has efficient and convenient functions for working with generators.
for client_id in itertools.islice(train_fd.client_ids(), 3):
  print('client_id =', client_id)
  print('# examples =', train_fd.client_size(client_id))

num_clients = 715
client_id = b'00192e4d5c9c3a5b:ALL_S_WELL_THAT_ENDS_WELL_CENTURION'
# examples = 5
client_id = b'004309f15562402e:THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_CAMPEIUS'
# examples = 13
client_id = b'00b20765b748920d:THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_ALL'
# examples = 15


As we notice, the client ids start with a random set of bits. This is to ensure that one can serially process clients and still get a random deterministic order, which is helpful in reading large datasets.

### Client datasets

We can query the dataset for a client from a federated dataset using their client ID and `fedjax.FederatedData.get_client()`. The output of `fedjax.FederatedData.get_client()` is a `fedjax.ClientDataset`. A `fedjax.ClientDataset` object

-   Stores all the examples for a given client and any preprocessing that should be applied.
-   Provides methods for batching, shuffling, and iterating over preprocessed examples.

In other words, **ClientDataset = examples + preprocessor**.

In [ ]:
client_id = b'105f96df763d4ddf:ALL_S_WELL_THAT_ENDS_WELL_GUIDERIUS_AND_ARVIRAGUS'
client_dataset = train_fd.get_client(client_id)
print(client_dataset)

FedJAX makes the assumption that individual client datasets are small and can easily fit in memory. This assumption is also reflected in many of FedJAX's design decisions. The examples in a client dataset can be viewed as a table, where the rows are
the individual examples, and the columns are the features (labels are viewed as
a feature in this context).

FedJAX uses a column based representation when loading a dataset into memory.

-   Each column is a NumPy array `x` of rank at least 1, where `x[i, ...]` is
    the value of this feature for the `i`-th example.
-   The complete set of examples is a dict-like object, from `str` feature
    names, to the corresponding column values.

Traditionally, a row based representation is used for representing the entire
dataset, and a column based representation is used for a single batch.

**In the context of federated learning, an individual client dataset is small
enough to easily fit into memory so the same representation is used for the
entire dataset and a batch.**

The examples of a single client can be viewed by calling `all_examples()`.

In [ ]:
client_dataset.all_examples()

{'x': array([[ 1, 55, 67, 84, 67, 47,  7, 67],
        [48, 16, 13, 32, 33, 14, 11, 78],
        [76, 78, 33, 19, 16, 66, 47,  3],
        [16, 27, 67, 23, 26, 47,  3, 27],
        [16,  7,  4, 67, 16, 51, 48, 68],
        [ 7, 26, 47, 27, 42, 16,  7,  4],
        [67, 72, 16, 48, 67, 27, 23, 71],
        [67, 65, 29, 79, 76, 51, 74, 12],
        [75, 54, 74, 19, 16, 66, 47,  3],
        [16, 67,  8, 67, 71, 47,  7, 61],
        [16, 14,  4, 67, 47, 16, 48, 67],
        [84, 67, 47,  7, 67, 48, 16, 12],
        [78, 29, 75, 78, 33, 16, 66, 47],
        [ 3, 16, 75, 73, 29, 11, 75, 76],
        [32, 19, 65, 16, 16, 16, 16, 16],
        [16, 16, 16, 16, 16, 16, 16, 16],
        [16, 16, 16, 16, 16, 16, 16, 16],
        [28, 68,  7,  4, 16, 75, 76, 32],
        [30, 74, 54, 65,  0,  0,  0,  0]], dtype=int32),
 'y': array([[55, 67, 84, 67, 47,  7, 67, 48],
        [16, 13, 32, 33, 14, 11, 78, 76],
        [78, 33, 19, 16, 66, 47,  3, 16],
        [27, 67, 23, 26, 47,  3, 27, 16],
        [

For Shakespeare, we are training a character-level language model, where the task is next character prediction, so the features are:

- `x` is a list of right-shifted sentences, e.g. `sentence[:-1]`
- `y` is a list of left-shifted sentences, e.g. `sentence[1:]`

This way, `y[i][j]` corresponds to the next character after `x[i][j]`.

In [ ]:
examples = client_dataset.all_examples()
print('x', examples['x'][0])
print('y', examples['y'][0])

x [ 1 55 67 84 67 47  7 67]
y [55 67 84 67 47  7 67 48]


However, you probably noticed that `x` and `y` are arrays of integers not text. This is because  `fedjax.datasets.shakespeare.load_data()` does some minimal preprocessing, such as a simple character look up that mapped characters to integer IDs. Later, we'll go over how this preprocessing was applied and how to add your own custom preprocessing.

For comparison, here's the unprocessed version of the same client dataset.

In [ ]:
# Unlike load_data(), load_split() always loads a single unprocessed split.
raw_fd = fedjax.datasets.shakespeare.load_split('train')
raw_fd.get_client(client_id).all_examples()

Reusing cached file '/tmp/.cache/fedjax/shakespeare_train.sqlite'


{'snippets': array([b'Re-enter POSTHUMUS, and seconds the Britons; they rescue\nCYMBELINE, and exeunt. Then re-enter LUCIUS and IACHIMO,\n                     with IMOGEN\n'],
       dtype=object)}

## Accessing client datasets from `fedjax.FederatedData`

`fedjax.FederatedData.get_client()` works well for querying data for a *single* client for exploring the dataset. However, we often want to query for multiple client datasets at the same time. In most FL algorithms, tens to hundreds of clients particpate in each training round. If we are not careful, our code can spend a lot of time loading data, leaving the accelerators (GPU or TPU) to idle.

In light of this, `fedjax.FederatedData` offers more efficient methods for querying multiple client datasets.

**We'll go through each access method from the MOST efficient to the LEAST efficient.**

### `clients()` and `shuffled_clients()`

**Fastest** sequential read friendly access. As we stated earlier, the client ids are appended with random bits. Hene, even sequential reads will go over clients in a pseudo-random order.

In [ ]:
# clients() and shuffled_clients() are sequential read friendly.
clients = train_fd.clients()
shuffled_clients = train_fd.shuffled_clients(buffer_size=100, seed=0)
print('clients =', clients)
print('shuffled_clients =', shuffled_clients)

clients = <generator object SQLiteFederatedData.clients at 0x7f346e2da850>
shuffled_clients = <generator object SQLiteFederatedData.shuffled_clients at 0x7f340f689b50>


They are generators, so we iterate over them to get the individual client datasets as tuples of (client_id, client_dataset).

`clients()` returns clients in an unspecified deterministic order. It is useful for going over the entire federated dataset for evaluation.

In [ ]:
# We use itertools.islice to select first three clients.
for client_id, client_dataset in itertools.islice(clients, 3):
  print('client_id =', client_id)
  print('# examples =', len(client_dataset))

client_id = b'00192e4d5c9c3a5b:ALL_S_WELL_THAT_ENDS_WELL_CENTURION'
# examples = 53
client_id = b'004309f15562402e:THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_CAMPEIUS'
# examples = 234
client_id = b'00b20765b748920d:THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_ALL'
# examples = 79


`shuffled_clients()` provides a stream of infinitely repeating shuffled client datasets, using buffered shuffling. It is suitable for training rounds where a nearly random shuffling is good enough.

In [ ]:
print('shuffled_clients()')
for client_id, client_dataset in itertools.islice(shuffled_clients, 3):
  print('client_id =', client_id)
  print('# examples =', len(client_dataset))

shuffled_clients()
client_id = b'0a18c2501d441fef:THE_TRAGEDY_OF_KING_LEAR_FLUTE'
# examples = 115
client_id = b'136c5586b7271525:THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_GLOUCESTER'
# examples = 3804
client_id = b'0d642a9b4bb27187:THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_MESSENGER'
# examples = 775


### `get_clients()`

**Slower** than `clients()` since it requires random read but uses prefetching to hide the latency of random read access. This also returns a generator of tuples of (client_id, client_dataset), in the order of the input client_ids.

In [ ]:
client_ids = [
    b'1db830204507458e:THE_TAMING_OF_THE_SHREW_SEBASTIAN',
    b'140784b36d08efbc:PERICLES__PRINCE_OF_TYRE_GHOST_OF_VAUGHAN',
    b'105f96df763d4ddf:ALL_S_WELL_THAT_ENDS_WELL_GUIDERIUS_AND_ARVIRAGUS'
]
for client_id, client_dataset in train_fd.get_clients(client_ids):
  print('client_id =', client_id)
  print('# examples =', len(client_dataset))

client_id = b'1db830204507458e:THE_TAMING_OF_THE_SHREW_SEBASTIAN'
# examples = 483
client_id = b'140784b36d08efbc:PERICLES__PRINCE_OF_TYRE_GHOST_OF_VAUGHAN'
# examples = 5
client_id = b'105f96df763d4ddf:ALL_S_WELL_THAT_ENDS_WELL_GUIDERIUS_AND_ARVIRAGUS'
# examples = 19


### `get_client()`

**Slowest** way of accessing client datasets. We usually reserve this method only for interactive exploration of a small number of clients.

In [ ]:
client_id = b'1db830204507458e:THE_TAMING_OF_THE_SHREW_SEBASTIAN'
print('client_id =', client_id)
print('# examples =', len(train_fd.get_client(client_id)))

client_id = b'1db830204507458e:THE_TAMING_OF_THE_SHREW_SEBASTIAN'
# examples = 483


## Batching client datasets

Next we'll go over different methods of iterating over a `fedjax.ClientDataset` as batched examples. All the following methods can be invoked in 2 ways:

-   Using a hyperparams object: This is the recommended way in library code. `batch_fn(hparams)`.
-   Using keyword arguments: The keyword arguments are used to construct a new hyperparams object, or override an existing one. `batch_fn(batch_size=2)` or `batch_fn(hparams, batch_size=2)` to override `batch_size`.

For the most part, we'll use method 2 for this colab, but method 1 is more suitable for writing library code.

In [ ]:
client_id = b'105f96df763d4ddf:ALL_S_WELL_THAT_ENDS_WELL_GUIDERIUS_AND_ARVIRAGUS'
client_dataset = train_fd.get_client(client_id)

### `batch()` for illustrations

Produces preprocessed batches in a fixed sequential order.

The final batch may contain fewer than `batch_size` examples. If used directly,
that may result in a large number of JIT recompilations. **Therefore we
should use `padded_batch()` or `shuffle_repeat_batch()` instead in most scenarios.**

In [ ]:
batches = list(client_dataset.batch(batch_size=8))
batches[-1]

{'x': array([[16, 16, 16, 16, 16, 16, 16, 16],
        [28, 68,  7,  4, 16, 75, 76, 32],
        [30, 74, 54, 65,  0,  0,  0,  0]], dtype=int32),
 'y': array([[16, 16, 16, 16, 16, 16, 16, 28],
        [68,  7,  4, 16, 75, 76, 32, 30],
        [74, 54, 65,  2,  0,  0,  0,  0]], dtype=int32)}

### `padded_batch()` for evaluation

Produces preprocessed padded batches in a fixed sequential order **for evaluation**.

When the number of examples in the dataset is not a multiple of `batch_size`,
the final batch may be smaller than `batch_size`. This may lead to [a large
number of JIT recompilations](https://jax.readthedocs.io/en/latest/jax-101/02-jitting.html). This can be circumvented by padding the final
batch to a small number of fixed sizes controlled by `num_batch_size_buckets`.

In [ ]:
# use list() to consume generator and store in memory.
padded_batches = list(client_dataset.padded_batch(batch_size=8, num_batch_size_buckets=3))
print('# batches =', len(padded_batches))
padded_batches[0]

# batches = 3


{'__mask__': array([ True,  True,  True,  True,  True,  True,  True,  True]),
 'x': array([[ 1, 55, 67, 84, 67, 47,  7, 67],
        [48, 16, 13, 32, 33, 14, 11, 78],
        [76, 78, 33, 19, 16, 66, 47,  3],
        [16, 27, 67, 23, 26, 47,  3, 27],
        [16,  7,  4, 67, 16, 51, 48, 68],
        [ 7, 26, 47, 27, 42, 16,  7,  4],
        [67, 72, 16, 48, 67, 27, 23, 71],
        [67, 65, 29, 79, 76, 51, 74, 12]], dtype=int32),
 'y': array([[55, 67, 84, 67, 47,  7, 67, 48],
        [16, 13, 32, 33, 14, 11, 78, 76],
        [78, 33, 19, 16, 66, 47,  3, 16],
        [27, 67, 23, 26, 47,  3, 27, 16],
        [ 7,  4, 67, 16, 51, 48, 68,  7],
        [26, 47, 27, 42, 16,  7,  4, 67],
        [72, 16, 48, 67, 27, 23, 71, 67],
        [65, 29, 79, 76, 51, 74, 12, 75]], dtype=int32)}

All batches contain an extra bool feature keyed by `'__mask__'`.
`batch['__mask__'][i]` tells us whether the `i`-th example in this batch
is an actual example (`batch['__mask__'][i] == True`), or a padding
example (`batch['__mask__'][i] == False`).

We repeatedly halve the batch size up to `num_batch_size_buckets - 1` times, until
we find the smallest one that is also >= the size of the final batch. Therefore
if `batch_size < 2^num_batch_size_buckets`, fewer bucket sizes will be actually
used. This will be seen when we look at the final batch that only has 4 examples when the original batch size was 8.

In [ ]:
padded_batches[-1]

{'__mask__': array([ True,  True,  True, False]),
 'x': array([[16, 16, 16, 16, 16, 16, 16, 16],
        [28, 68,  7,  4, 16, 75, 76, 32],
        [30, 74, 54, 65,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0]], dtype=int32),
 'y': array([[16, 16, 16, 16, 16, 16, 16, 28],
        [68,  7,  4, 16, 75, 76, 32, 30],
        [74, 54, 65,  2,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0]], dtype=int32)}

### `shuffle_repeat_batch()` for training

Produces preprocessed batches in a shuffled and repeated order **for training**.

Shuffling is done without replacement, therefore for a dataset of N examples,
the first `ceil(N/batch_size)` batches are guarranteed to cover the entire
dataset. Unlike `batch()` or `padded_batch()`, batches from
`shuffle_repeat_batch()` always contain exactly `batch_size` examples. Also
unlike TensorFlow, that holds even when `drop_remainder=False`.

By default the iteration stops after the first epoch.

In [ ]:
print('# batches')
len(list(client_dataset.shuffle_repeat_batch(batch_size=8)))

# batches


3

The number of batches produced from the iteration can be controlled by the `(num_epochs, num_steps,
drop_remainder)` combination:

If both `num_epochs` and `num_steps` are None, the shuffle-repeat process continues forever.


In [ ]:
infinite_bs = client_dataset.shuffle_repeat_batch(
    batch_size=8, num_epochs=None, num_steps=None)
for i, b in zip(range(6), infinite_bs):
  print(i)

0
1
2
3
4
5


If `num_epochs` is set and `num_steps` is None, as few batches as needed to go
over the dataset this many passes are produced. Further,

-   If `drop_remainder` is False (the default), the final batch is filled with
    additionally sampled examples to contain `batch_size` examples.
-   If `drop_remainder` is True, the final batch is dropped if it contains fewer
    than `batch_size` examples. This may result in examples being skipped when
    `num_epochs=1`.

In [ ]:
print('# batches w/ drop_remainder=False')
print(len(list(client_dataset.shuffle_repeat_batch(batch_size=8, num_epochs=1, num_steps=None))))
print('# batches w/ drop_remainder=True')
print(len(list(client_dataset.shuffle_repeat_batch(batch_size=8, num_epochs=1, num_steps=None, drop_remainder=True))))

# batches w/ drop_remainder=False
3
# batches w/ drop_remainder=True
2


If `num_steps` is set and `num_steps` is None, exactly this many batches are
produced. `drop_remainder` has no effect in this case.

In [ ]:
print('# batches w/ num_steps set and drop_remainder=True')
print(len(list(client_dataset.shuffle_repeat_batch(batch_size=8, num_epochs=None, num_steps=3, drop_remainder=True))))

# batches w/ num_steps set and drop_remainder=True
3


If both `num_epochs` and `num_steps` are set, the fewer number of batches
between the two conditions are produced.

In [ ]:
print('# batches w/ num_epochs and num_steps set')
print(len(list(client_dataset.shuffle_repeat_batch(batch_size=8, num_epochs=1, num_steps=6))))

# batches w/ num_epochs and num_steps set
3


If reproducible iteration order is desired, a fixed `seed` can be used. When
`seed` is None, repeated iteration over the same object may produce batches in a
different order.

In [ ]:
# Random shuffling.
print(list(client_dataset.shuffle_repeat_batch(batch_size=2, seed=None))[0])
# Fixed shuffling.
print(list(client_dataset.shuffle_repeat_batch(batch_size=2, seed=0))[0])

{'x': array([[30, 74, 54, 65,  0,  0,  0,  0],
       [75, 54, 74, 19, 16, 66, 47,  3]], dtype=int32), 'y': array([[74, 54, 65,  2,  0,  0,  0,  0],
       [54, 74, 19, 16, 66, 47,  3, 16]], dtype=int32)}
{'x': array([[16, 14,  4, 67, 47, 16, 48, 67],
       [48, 16, 13, 32, 33, 14, 11, 78]], dtype=int32), 'y': array([[14,  4, 67, 47, 16, 48, 67, 84],
       [16, 13, 32, 33, 14, 11, 78, 76]], dtype=int32)}


## Preprocessing

Preprocessing can be done at two levels:

1.  The client dataset level with `fedjax.ClientPreprocessor`
2.  The batch level with `fedjax.BatchPreprocessor`

**Examples of preprocessing possible at either the client dataset level, or
the batch level**

Such preprocessing is deterministic, and strictly per-example:

- Casting a feature from `int8` to `float32`
- Adding a new feature derived from existing features
- Removing a feature (although the better place to do so is at the dataset
  level)

A simple rule for deciding where to carry out the preprocessing in this case
is the following,

- Does this make batching cheaper (e.g. removing features)? If so, do it at
  the dataset level.
- Otherwise, do it at the batch level.

Assuming preprocessing time is linear in the number of examples, preprocessing
at the batch level has the benefit of evenly distributing host compute work,
which may overlap better with asynchronous JAX compute work on GPU/TPU.

**Examples of preprocessing only possible at the batch level**

- Data augmentation (e.g. random cropping).
- Padding at the batch size dimension.

**Examples of preprocessing only possible at the dataset level**

- Those that require knowing the client id.
- Capping the number of examples.
- Altering what it means to be an example: e.g. in certain language model
  setups, sentences are concatenated and then split into equal sized chunks.

In [ ]:
# Load unpreprocessed data.
raw_fd = fedjax.datasets.shakespeare.load_split('train')

Reusing cached file '/tmp/.cache/fedjax/shakespeare_train.sqlite'


### Registering preprocessing

We can register extra preprocessing on the `fedjax.FederatedData`
using `preprocess_client()` and `preprocess_batch()` for the client dataset
level and batch level, respectively. `fedjax.FederatedData` then passes information about preprocessing on to a `fedjax.ClientDataset`.

Below, we will walk through an example preprocessing pipeline for Shakespeare
that turns text into sequences of integer labels. First, we need a look-up table.

In [ ]:
def _build_look_up_table(vocab, num_reserved):
  """Builds a look-up table from a byte to its integer label."""
  oov = num_reserved + len(vocab)
  vocab_size = oov + 1
  table = np.full([256], oov, dtype=np.int32)
  for i, c in enumerate(vocab):
    table[c] = num_reserved + i
  return table, vocab_size


# Vocabulary re-used from the Federated Learning for Text Generation tutorial.
# https://www.tensorflow.org/federated/tutorials/federated_learning_for_text_generation
TABLE, VOCAB_SIZE = _build_look_up_table(
    b'dhlptx@DHLPTX $(,048cgkoswCGKOSW[_#\'/37;?bfjnrvzBFJNRVZ"&*.26:\naeimquyAEIMQUY]!%)-159\r',
    num_reserved=3)
OOV = VOCAB_SIZE - 1
# Reserved labels.
PAD = 0
BOS = 1
EOS = 2

All snippets in a client dataset are first joined into a single sequence (with
BOS/EOS added), and then split into pairs of `sequence_length` chunks for
language model training. For example, with sequence_length=3, `[b'ABCD', b'E']`
becomes

```
Input sequences:  [[BOS, A, B], [C, D, EOS],   [BOS, E, PAD]]
Output seqeunces: [[A, B, C],   [D, EOS, BOS], [E, EOS, PAD]]
```

In [ ]:
def preprocess_client(client_id, examples, sequence_length):
  """Turns snippets into sequences of integer labels."""
  del client_id
  snippets = examples['snippets']
  # Join all snippets into a single label sequence.
  joined_length = sum(len(i) + 2 for i in snippets)
  joined = np.zeros([joined_length], dtype=np.int32)
  offset = 0
  for i in snippets:
    joined[offset] = BOS
    joined[offset + 1:offset + 1 + len(i)] = TABLE[list(i)]
    joined[offset + 1 + len(i)] = EOS
    offset += len(i) + 2
  # Split into input/output sequences of size `sequence_length`.
  padded_length = ((joined_length - 1 + sequence_length - 1) //
                   sequence_length * sequence_length)
  input_labels = np.full([padded_length], PAD, dtype=np.int32)
  input_labels[:joined_length - 1] = joined[:-1]
  output_labels = np.full([padded_length], PAD, dtype=np.int32)
  output_labels[:joined_length - 1] = joined[1:]
  return {
      'x': input_labels.reshape([-1, sequence_length]),
      'y': output_labels.reshape([-1, sequence_length])
  }

Given N input snippets as a bytes NumPy array, the output features will be (M below is possibly different from N):

-   `x`: [M, sequence_length] int32 input labels, in the range of [0,
    shakespeare.VOCAB_SIZE)
-   `y`: [M, sequence_length] int32 output labels, in the range of [0,
    shakespeare.VOCAB_SIZE)

In [ ]:
client_id = b'105f96df763d4ddf:ALL_S_WELL_THAT_ENDS_WELL_GUIDERIUS_AND_ARVIRAGUS'
raw_client_dataset = raw_fd.get_client(client_id)
print('Raw unprocessed client dataset')
print(raw_client_dataset.all_examples())

preprocess = functools.partial(preprocess_client, sequence_length=10)
# A new FederatedData object is created when we register new preprocessing.
processed_fd = raw_fd.preprocess_client(preprocess)
client_dataset = processed_fd.get_client(client_id)
print('Preprocessed client dataset')
print(client_dataset.all_examples())

Raw unprocessed client dataset
{'snippets': array([b'Re-enter POSTHUMUS, and seconds the Britons; they rescue\nCYMBELINE, and exeunt. Then re-enter LUCIUS and IACHIMO,\n                     with IMOGEN\n'],
      dtype=object)}
Preprocessed client dataset
{'x': array([[ 1, 55, 67, 84, 67, 47,  7, 67, 48, 16],
       [13, 32, 33, 14, 11, 78, 76, 78, 33, 19],
       [16, 66, 47,  3, 16, 27, 67, 23, 26, 47],
       [ 3, 27, 16,  7,  4, 67, 16, 51, 48, 68],
       [ 7, 26, 47, 27, 42, 16,  7,  4, 67, 72],
       [16, 48, 67, 27, 23, 71, 67, 65, 29, 79],
       [76, 51, 74, 12, 75, 54, 74, 19, 16, 66],
       [47,  3, 16, 67,  8, 67, 71, 47,  7, 61],
       [16, 14,  4, 67, 47, 16, 48, 67, 84, 67],
       [47,  7, 67, 48, 16, 12, 78, 29, 75, 78],
       [33, 16, 66, 47,  3, 16, 75, 73, 29, 11],
       [75, 76, 32, 19, 65, 16, 16, 16, 16, 16],
       [16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [16, 16, 16, 16, 16, 16, 28, 68,  7,  4],
       [16, 75, 76, 32, 30, 74, 54, 65,  0,  0]], dt

### `BatchPreprocessor`

Preprocessing on a batch of examples can be easily done via a chain of
functions. A `fedjax.BatchPreprocessor` holds a chain of preprocessing functions, and applies
them in order on batched examples. Each individual preprocessing function
operates over multiple examples, instead of just 1 example.

In [ ]:
preprocessor = fedjax.BatchPreprocessor([
  # Flattens `pixels`.
  lambda x: {**x, 'pixels': x['pixels'].reshape([-1, 28 * 28])},
  # Introduce `binary_label`.
  lambda x: {**x, 'binary_label': x['label'] % 2},
])
fake_emnist = {
  'pixels': np.random.RandomState(0).uniform(size=(2, 28, 28)),
  'label': np.random.RandomState(1).randint(10, size=(2,))
}
preprocessor(fake_emnist)
# Produces a dict of [2, 28*28] "pixels", [2,] "label" and "binary_label".

{'binary_label': array([1, 0]),
 'label': array([5, 8]),
 'pixels': array([[0.5488135 , 0.71518937, 0.60276338, ..., 0.42690436, 0.84285489,
         0.81803331],
        [0.10241376, 0.15638335, 0.30419869, ..., 0.34010075, 0.06859683,
         0.2289076 ]])}

Given a `fedjax.BatchPreprocessor`, a new `fedjax.BatchPreprocessor` can be
created with an additional preprocessing function appended to the chain.

In [ ]:
# Continuing from the previous example.
new_preprocessor = preprocessor.append(
  lambda x: {**x, 'sum_pixels': np.sum(x['pixels'], axis=1)})
new_preprocessor(fake_emnist)
# Produces a dict of [2, 28*28] "pixels", [2,] "sum_pixels", "label" and
# "binary_label".

{'binary_label': array([1, 0]),
 'label': array([5, 8]),
 'pixels': array([[0.5488135 , 0.71518937, 0.60276338, ..., 0.42690436, 0.84285489,
         0.81803331],
        [0.10241376, 0.15638335, 0.30419869, ..., 0.34010075, 0.06859683,
         0.2289076 ]]),
 'sum_pixels': array([388.36415218, 401.91274905])}

The main difference of this preprocessor and `fedjax.ClientPreprocessor` is that
`fedjax.ClientPreprocessor` also takes `client_id` as input. Because of the
identical representation between batched examples and all examples in a client
dataset, certain preprocessing can be done with either
`fedjax.BatchPreprocessor` or `fedjax.ClientPreprocessor`.

### `ClientPreprocessor`

A chain of preprocessing functions on all examples of a client dataset.

This is very similar to `fedjax.BatchPreprocessor`, with the main difference
being that `ClientPreprocessor` also takes `client_id` as input.

In [ ]:
preprocessor = fedjax.ClientPreprocessor([
  # Adds `client_id_length`.
  lambda cid, x: {**x, 'client_id_length': np.ones_like(x['label']) * len(cid)}
])
fake_emnist = {
  'label': np.random.RandomState(0).randint(10, size=(2,))
}
client_id = b'123456'
preprocessor(client_id, fake_emnist)
# Produces a dict of [2,] "label" and "client_id_length".

{'client_id_length': array([6, 6]), 'label': array([5, 0])}

## Creating custom federated datasets

In many scenarios, it is desirable to create a custom federated dataset. FedJAX provides `fedjax.InMemoryDataset` to create small custom datasets. `fedjax.InMemoryDataset` takes a dictionary of numpy examples keyed by client id and creates a `fedjax.FederatedData` that is compatible with the rest of the library. We illustrate with a simple example.

In [ ]:
# Obtain MNIST dataset from tensorflow and convert to numpy format.
import tensorflow_datasets as tfds
(ds_train, ds_test) = tfds.load('mnist',
                                split=['train', 'test'],
                                shuffle_files=True,
                                as_supervised=True,
                                with_info=False)
features, labels = list(ds_train.batch(60000).as_numpy_iterator())[0]
print('features shape', features.shape)
print('labels shape', labels.shape)

# Randomly split dataset into 100 clients and load them to a dictionary.
indices = np.random.randint(100, size=60000)
client_id_to_dataset_mapping = {}
for i in range(100):
  client_id_to_dataset_mapping[i] = {'x': features[indices==i, :, : , :],
                                     'y': labels[indices==i]}

# Create fedjax.InMemoryDataset.
iid_mnist_federated_data = fedjax.InMemoryFederatedData(
    client_id_to_dataset_mapping)

print('number of clients in iid_mnist_data',
      iid_mnist_federated_data.num_clients())

features shape (60000, 28, 28, 1)
labels shape (60000,)
number of clients in iid_mnist_data 100


## Recap

In this tutorial, we have covered the following:

1. Using `fedjax.FederatedData`.
2. Different ways of batching client datasets.
3. Different ways of processing client datasets.
4. Creating small custom federated datasets. 